In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)

# Load data
print("="*80)
print("LOADING DATA")
print("="*80)
df = pd.read_csv('../data/NSI.csv')
print(f"Original dataset shape: {df.shape}")
print(f"Date range: {df['REPORT_YEAR'].min()}-{df['REPORT_MONTH'].min()} to {df['REPORT_YEAR'].max()}-{df['REPORT_MONTH'].max()}")

# Create datetime column and aggregate by month
df['date'] = pd.to_datetime({'year': df['REPORT_YEAR'], 
                              'month': df['REPORT_MONTH'], 
                              'day': 1})

# Aggregate by month - NSI is the target (averaged across neighborhoods)
# Also include crime metrics as potential context
ts_data = df.groupby('date').agg({
    'NSI': 'mean',  # TARGET VARIABLE
    'TotalCrimeScore': 'sum',
    'Crime_Count': 'sum',
    'Prev_Month_NSI': 'mean',
    'NSI_3M_Avg': 'mean'
}).reset_index()

ts_data.set_index('date', inplace=True)
ts_data = ts_data.sort_index()

print(f"\nAggregated time series shape: {ts_data.shape}")
print(f"Time series range: {ts_data.index.min()} to {ts_data.index.max()}")
print(f"Frequency: {ts_data.index.freq}")

# Display summary statistics
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)
print(ts_data.describe())

LOADING DATA
Original dataset shape: (21910, 10)
Date range: 2014-1 to 2025-12

Aggregated time series shape: (138, 5)
Time series range: 2014-04-01 00:00:00 to 2025-09-01 00:00:00
Frequency: None

SUMMARY STATISTICS
              NSI  TotalCrimeScore  Crime_Count  Prev_Month_NSI  NSI_3M_Avg
count  138.000000       138.000000   138.000000      138.000000  138.000000
mean     0.854593     11218.681159  3229.594203        0.854910    0.855366
std      0.020579      1577.159601   519.397667        0.020694    0.019712
min      0.804047      7635.000000  2109.000000        0.804047    0.809916
25%      0.839856     10194.000000  2857.750000        0.839856    0.841679
50%      0.858576     10895.500000  3116.500000        0.858847    0.858340
75%      0.867605     12355.750000  3594.500000        0.868112    0.869156
max      0.901205     15083.000000  4505.000000        0.901656    0.889467


In [25]:
# ============================================================================
# EXPLORATORY DATA ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("EXPLORATORY DATA ANALYSIS")
print("="*80)

# Plot 1: Time series of NSI (target) and related metrics
fig, axes = plt.subplots(3, 1, figsize=(15, 10))

# NSI (Neighborhood Safety Index) - TARGET
axes[0].plot(ts_data.index, ts_data['NSI'], linewidth=1.5, color='darkgreen')
axes[0].set_title('NSI (Neighborhood Safety Index) Over Time - TARGET VARIABLE', fontsize=14, fontweight='bold')
axes[0].set_ylabel('NSI', fontsize=12)
axes[0].grid(True, alpha=0.3)

# Total Crime Score (context)
axes[1].plot(ts_data.index, ts_data['TotalCrimeScore'], linewidth=1.5, color='darkred')
axes[1].set_title('Total Crime Score Over Time (for reference)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Total Crime Score', fontsize=12)
axes[1].grid(True, alpha=0.3)

# Crime Count (context)
axes[2].plot(ts_data.index, ts_data['Crime_Count'], linewidth=1.5, color='darkblue')
axes[2].set_title('Total Crime Count Over Time (for reference)', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Crime Count', fontsize=12)
axes[2].set_xlabel('Date', fontsize=12)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../arima/01_time_series_overview.png', dpi=300, bbox_inches='tight')
print("✓ Saved: 01_time_series_overview.png")
plt.close()

# Plot 2: Seasonal decomposition
print("\nPerforming seasonal decomposition...")
decomposition = seasonal_decompose(ts_data['NSI'], model='additive', period=12)

fig, axes = plt.subplots(4, 1, figsize=(15, 12))
decomposition.observed.plot(ax=axes[0], title='Observed', color='darkgreen')
decomposition.trend.plot(ax=axes[1], title='Trend', color='darkblue')
decomposition.seasonal.plot(ax=axes[2], title='Seasonal', color='darkorange')
decomposition.resid.plot(ax=axes[3], title='Residual', color='darkred')

for ax in axes:
    ax.grid(True, alpha=0.3)
    ax.set_ylabel('Value')
    
axes[3].set_xlabel('Date')
fig.suptitle('Seasonal Decomposition of NSI (Neighborhood Safety Index)', fontsize=16, fontweight='bold', y=1.001)
plt.tight_layout()
plt.savefig('../arima/02_seasonal_decomposition.png', dpi=300, bbox_inches='tight')
print("✓ Saved: 02_seasonal_decomposition.png")
plt.close()


EXPLORATORY DATA ANALYSIS
✓ Saved: 01_time_series_overview.png

Performing seasonal decomposition...
✓ Saved: 02_seasonal_decomposition.png


In [26]:
# ============================================================================
# STATIONARITY TESTS
# ============================================================================

print("\n" + "="*80)
print("STATIONARITY ANALYSIS")
print("="*80)

def adf_test(series, name=''):
    """Perform Augmented Dickey-Fuller test"""
    result = adfuller(series.dropna())
    print(f'\n{name}')
    print(f'ADF Statistic: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    print(f'Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value:.3f}')
    if result[1] <= 0.05:
        print("→ Series is STATIONARY (reject H0)")
    else:
        print("→ Series is NON-STATIONARY (fail to reject H0)")
    return result[1]

# Test original series
p_value_original = adf_test(ts_data['NSI'], 'Original Series (NSI)')

# Test first difference
ts_data['NSI_diff1'] = ts_data['NSI'].diff()
p_value_diff1 = adf_test(ts_data['NSI_diff1'], 'First Difference')

# Test seasonal difference
ts_data['NSI_seasonal_diff'] = ts_data['NSI'].diff(12)
p_value_seasonal = adf_test(ts_data['NSI_seasonal_diff'], 'Seasonal Difference (lag=12)')

# Plot ACF and PACF
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Original series
plot_acf(ts_data['NSI'].dropna(), lags=40, ax=axes[0, 0])
axes[0, 0].set_title('ACF: Original Series (NSI)', fontsize=12, fontweight='bold')
plot_pacf(ts_data['NSI'].dropna(), lags=40, ax=axes[0, 1])
axes[0, 1].set_title('PACF: Original Series (NSI)', fontsize=12, fontweight='bold')

# First differenced series
plot_acf(ts_data['NSI_diff1'].dropna(), lags=40, ax=axes[1, 0])
axes[1, 0].set_title('ACF: First Difference', fontsize=12, fontweight='bold')
plot_pacf(ts_data['NSI_diff1'].dropna(), lags=40, ax=axes[1, 1])
axes[1, 1].set_title('PACF: First Difference', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('../arima/03_acf_pacf_plots.png', dpi=300, bbox_inches='tight')
print("\n✓ Saved: 03_acf_pacf_plots.png")
plt.close()


STATIONARITY ANALYSIS

Original Series (NSI)
ADF Statistic: -2.097649
p-value: 0.245461
Critical Values:
   1%: -3.485
   5%: -2.885
   10%: -2.579
→ Series is NON-STATIONARY (fail to reject H0)

First Difference
ADF Statistic: -2.255635
p-value: 0.186676
Critical Values:
   1%: -3.485
   5%: -2.885
   10%: -2.579
→ Series is NON-STATIONARY (fail to reject H0)

Seasonal Difference (lag=12)
ADF Statistic: -2.493161
p-value: 0.117147
Critical Values:
   1%: -3.490
   5%: -2.887
   10%: -2.581
→ Series is NON-STATIONARY (fail to reject H0)

✓ Saved: 03_acf_pacf_plots.png


In [27]:
# ============================================================================
# TRAIN-TEST SPLIT
# ============================================================================

print("\n" + "="*80)
print("TRAIN-TEST SPLIT")
print("="*80)

# Use 80% for training, 20% for testing
train_size = int(len(ts_data) * 0.8)
train_data = ts_data['NSI'][:train_size]
test_data = ts_data['NSI'][train_size:]

print(f"Training set: {train_data.index.min()} to {train_data.index.max()} ({len(train_data)} observations)")
print(f"Test set: {test_data.index.min()} to {test_data.index.max()} ({len(test_data)} observations)")


TRAIN-TEST SPLIT
Training set: 2014-04-01 00:00:00 to 2023-05-01 00:00:00 (110 observations)
Test set: 2023-06-01 00:00:00 to 2025-09-01 00:00:00 (28 observations)


In [28]:
# ============================================================================
# SARIMA MODEL FITTING
# ============================================================================

print("\n" + "="*80)
print("SARIMA MODEL TRAINING")
print("="*80)

# Define SARIMA parameters to test
# Based on ACF/PACF analysis and seasonal pattern (period=12)
# We'll test a few configurations

sarima_configs = [
    ((1, 1, 1), (1, 1, 1, 12)),  # Classic SARIMA
    ((2, 1, 2), (1, 1, 1, 12)),  # Higher order
    ((1, 1, 2), (1, 1, 1, 12)),  # Alternative
    ((0, 1, 1), (0, 1, 1, 12)),  # Simple model
    ((1, 1, 0), (1, 1, 0, 12)),  # AR only
]

best_aic = np.inf
best_model = None
best_order = None
results_list = []

print("\nTesting different SARIMA configurations...")
print("-" * 80)

for order, seasonal_order in sarima_configs:
    try:
        model = SARIMAX(train_data, 
                       order=order, 
                       seasonal_order=seasonal_order,
                       enforce_stationarity=False,
                       enforce_invertibility=False)
        fitted_model = model.fit(disp=False, maxiter=200)
        
        aic = fitted_model.aic
        bic = fitted_model.bic
        
        results_list.append({
            'Order': order,
            'Seasonal_Order': seasonal_order,
            'AIC': aic,
            'BIC': bic
        })
        
        print(f"SARIMA{order}x{seasonal_order}: AIC={aic:.2f}, BIC={bic:.2f}")
        
        if aic < best_aic:
            best_aic = aic
            best_model = fitted_model
            best_order = (order, seasonal_order)
            
    except Exception as e:
        print(f"SARIMA{order}x{seasonal_order}: Failed - {str(e)[:50]}")

print("-" * 80)
print(f"\nBest Model: SARIMA{best_order[0]}x{best_order[1]}")
print(f"AIC: {best_aic:.2f}")
print(f"\nModel Summary:")
print(best_model.summary())



SARIMA MODEL TRAINING

Testing different SARIMA configurations...
--------------------------------------------------------------------------------
SARIMA(1, 1, 1)x(1, 1, 1, 12): AIC=-526.23, BIC=-514.14
SARIMA(2, 1, 2)x(1, 1, 1, 12): AIC=-520.17, BIC=-503.33
SARIMA(1, 1, 2)x(1, 1, 1, 12): AIC=-519.11, BIC=-504.66
SARIMA(0, 1, 1)x(0, 1, 1, 12): AIC=-527.59, BIC=-520.33
SARIMA(1, 1, 0)x(1, 1, 0, 12): AIC=-519.41, BIC=-512.12
--------------------------------------------------------------------------------

Best Model: SARIMA(0, 1, 1)x(0, 1, 1, 12)
AIC: -527.59

Model Summary:
                                     SARIMAX Results                                      
Dep. Variable:                                NSI   No. Observations:                  110
Model:             SARIMAX(0, 1, 1)x(0, 1, 1, 12)   Log Likelihood                 266.793
Date:                            Mon, 01 Dec 2025   AIC                           -527.587
Time:                                    15:22:48   BIC

In [29]:
# ============================================================================
# MODEL DIAGNOSTICS
# ============================================================================

print("\n" + "="*80)
print("MODEL DIAGNOSTICS")
print("="*80)

# Plot diagnostics
fig = best_model.plot_diagnostics(figsize=(15, 10))
plt.tight_layout()
plt.savefig('../arima/04_model_diagnostics.png', dpi=300, bbox_inches='tight')
print("✓ Saved: 04_model_diagnostics.png")
plt.close()


MODEL DIAGNOSTICS
✓ Saved: 04_model_diagnostics.png


In [30]:
# ============================================================================
# PREDICTIONS AND EVALUATION
# ============================================================================

print("\n" + "="*80)
print("MODEL PREDICTIONS AND EVALUATION")
print("="*80)

# In-sample predictions (training set)
train_predictions = best_model.fittedvalues

# Out-of-sample predictions (test set)
forecast_steps = len(test_data)
forecast = best_model.forecast(steps=forecast_steps)
forecast_index = test_data.index

# Calculate metrics for test set
mse = mean_squared_error(test_data, forecast)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_data, forecast)
mape = np.mean(np.abs((test_data - forecast) / test_data)) * 100
r2 = r2_score(test_data, forecast)

print(f"\nTest Set Performance Metrics:")
print(f"{'='*50}")
print(f"RMSE:  {rmse:.2f}")
print(f"MAE:   {mae:.2f}")
print(f"MAPE:  {mape:.2f}%")
print(f"R²:    {r2:.4f}")

# Calculate metrics for training set
train_mse = mean_squared_error(train_data, train_predictions)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(train_data, train_predictions)
train_r2 = r2_score(train_data, train_predictions)

print(f"\nTraining Set Performance Metrics:")
print(f"{'='*50}")
print(f"RMSE:  {train_rmse:.2f}")
print(f"MAE:   {train_mae:.2f}")
print(f"R²:    {train_r2:.4f}")



MODEL PREDICTIONS AND EVALUATION

Test Set Performance Metrics:
RMSE:  0.03
MAE:   0.02
MAPE:  2.56%
R²:    -3.4735

Training Set Performance Metrics:
RMSE:  0.10
MAE:   0.02
R²:    -40.5285


In [31]:
# ============================================================================
# VISUALIZATION OF RESULTS
# ============================================================================

print("\n" + "="*80)
print("GENERATING VISUALIZATIONS")
print("="*80)

# Plot 1: Actual vs Predicted (Full Series)
fig, ax = plt.subplots(figsize=(16, 6))

# Plot training data
ax.plot(train_data.index, train_data.values, 
        label='Training Data', color='blue', linewidth=1.5, alpha=0.7)

# Plot test data
ax.plot(test_data.index, test_data.values, 
        label='Test Data', color='green', linewidth=1.5, alpha=0.7)

# Plot predictions on training set
ax.plot(train_data.index, train_predictions, 
        label='Training Predictions', color='orange', linewidth=1.5, alpha=0.8, linestyle='--')

# Plot forecast on test set
ax.plot(forecast_index, forecast, 
        label='Test Forecast', color='red', linewidth=2, alpha=0.9)

# Add confidence interval for forecast
forecast_df = best_model.get_forecast(steps=forecast_steps)
forecast_ci = forecast_df.conf_int()
ax.fill_between(forecast_index, 
                forecast_ci.iloc[:, 0], 
                forecast_ci.iloc[:, 1], 
                color='red', alpha=0.2, label='95% Confidence Interval')

ax.set_title(f'SARIMA{best_order[0]}x{best_order[1]}: Actual vs Predicted NSI', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('NSI (Neighborhood Safety Index)', fontsize=12)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../arima/05_actual_vs_predicted.png', dpi=300, bbox_inches='tight')
print("✓ Saved: 05_actual_vs_predicted.png")
plt.close()

# Plot 2: Zoomed in on Test Period
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(test_data.index, test_data.values, 
        label='Actual', color='green', linewidth=2, marker='o', markersize=4)
ax.plot(forecast_index, forecast, 
        label='Forecast', color='red', linewidth=2, marker='s', markersize=4)

ax.fill_between(forecast_index, 
                forecast_ci.iloc[:, 0], 
                forecast_ci.iloc[:, 1], 
                color='red', alpha=0.2, label='95% Confidence Interval')

ax.set_title('Test Set: Actual vs Forecast NSI (Detailed View)', fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('NSI (Neighborhood Safety Index)', fontsize=12)
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../arima/06_test_set_detail.png', dpi=300, bbox_inches='tight')
print("✓ Saved: 06_test_set_detail.png")
plt.close()

# Plot 3: Residuals Analysis
residuals = test_data.values - forecast

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Residuals over time
axes[0, 0].plot(forecast_index, residuals, color='darkred', linewidth=1.5)
axes[0, 0].axhline(y=0, color='black', linestyle='--', linewidth=1)
axes[0, 0].set_title('Residuals Over Time', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Residual')
axes[0, 0].grid(True, alpha=0.3)

# Histogram of residuals
axes[0, 1].hist(residuals, bins=15, color='darkblue', alpha=0.7, edgecolor='black')
axes[0, 1].set_title('Distribution of Residuals', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Residual')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].grid(True, alpha=0.3)

# Q-Q plot
from scipy import stats
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Scatter: Actual vs Predicted
axes[1, 1].scatter(test_data.values, forecast, alpha=0.6, color='purple', s=50)
axes[1, 1].plot([test_data.min(), test_data.max()], 
                [test_data.min(), test_data.max()], 
                'r--', linewidth=2, label='Perfect Prediction')
axes[1, 1].set_title('Actual vs Predicted', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Actual')
axes[1, 1].set_ylabel('Predicted')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../arima/07_residual_analysis.png', dpi=300, bbox_inches='tight')
print("✓ Saved: 07_residual_analysis.png")
plt.close()



GENERATING VISUALIZATIONS
✓ Saved: 05_actual_vs_predicted.png
✓ Saved: 06_test_set_detail.png
✓ Saved: 07_residual_analysis.png


In [32]:
# ============================================================================
# SAVE RESULTS
# ============================================================================

print("\n" + "="*80)
print("SAVING RESULTS")
print("="*80)

# Create results dataframe
results_df = pd.DataFrame({
    'Date': test_data.index,
    'Actual': test_data.values,
    'Predicted': forecast.values,
    'Residual': residuals,
    'Lower_CI': forecast_ci.iloc[:, 0].values,
    'Upper_CI': forecast_ci.iloc[:, 1].values
})

results_df.to_csv('../arima/sarima_predictions.csv', index=False)
print("✓ Saved: sarima_predictions.csv")

# Save model comparison results
comparison_df = pd.DataFrame(results_list)
comparison_df = comparison_df.sort_values('AIC')
comparison_df.to_csv('../arima/model_comparison.csv', index=False)
print("✓ Saved: model_comparison.csv")

# Create summary report
with open('../arima/sarima_summary.txt', 'w') as f:
    f.write("="*80 + "\n")
    f.write("SARIMA MODEL SUMMARY REPORT\n")
    f.write("="*80 + "\n\n")
    
    f.write("DATA INFORMATION\n")
    f.write("-"*80 + "\n")
    f.write(f"Total observations: {len(ts_data)}\n")
    f.write(f"Training set: {len(train_data)} observations ({train_data.index.min()} to {train_data.index.max()})\n")
    f.write(f"Test set: {len(test_data)} observations ({test_data.index.min()} to {test_data.index.max()})\n\n")
    
    f.write("BEST MODEL\n")
    f.write("-"*80 + "\n")
    f.write(f"Model: SARIMA{best_order[0]}x{best_order[1]}\n")
    f.write(f"AIC: {best_aic:.2f}\n")
    f.write(f"BIC: {best_model.bic:.2f}\n\n")
    
    f.write("PERFORMANCE METRICS - TEST SET\n")
    f.write("-"*80 + "\n")
    f.write(f"RMSE:  {rmse:.2f}\n")
    f.write(f"MAE:   {mae:.2f}\n")
    f.write(f"MAPE:  {mape:.2f}%\n")
    f.write(f"R²:    {r2:.4f}\n\n")
    
    f.write("PERFORMANCE METRICS - TRAINING SET\n")
    f.write("-"*80 + "\n")
    f.write(f"RMSE:  {train_rmse:.2f}\n")
    f.write(f"MAE:   {train_mae:.2f}\n")
    f.write(f"R²:    {train_r2:.4f}\n\n")

print("✓ Saved: sarima_summary.txt")

print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("="*80)
print(f"\nAll outputs saved to: ../arima/")
print(f"\nGenerated files:")
print("  - 01_time_series_overview.png")
print("  - 02_seasonal_decomposition.png")
print("  - 03_acf_pacf_plots.png")
print("  - 04_model_diagnostics.png")
print("  - 05_actual_vs_predicted.png")
print("  - 06_test_set_detail.png")
print("  - 07_residual_analysis.png")
print("  - sarima_predictions.csv")
print("  - model_comparison.csv")
print("  - sarima_summary.txt")


SAVING RESULTS
✓ Saved: sarima_predictions.csv
✓ Saved: model_comparison.csv
✓ Saved: sarima_summary.txt

ANALYSIS COMPLETE!

All outputs saved to: ../arima/

Generated files:
  - 01_time_series_overview.png
  - 02_seasonal_decomposition.png
  - 03_acf_pacf_plots.png
  - 04_model_diagnostics.png
  - 05_actual_vs_predicted.png
  - 06_test_set_detail.png
  - 07_residual_analysis.png
  - sarima_predictions.csv
  - model_comparison.csv
  - sarima_summary.txt
